In [65]:
from urllib.request import urlopen 
from bs4 import BeautifulSoup 
import pandas as pd
%run rookie_scraper.ipynb
import matplotlib.pyplot as plt
%matplotlib inline

rookie_scraper.ipynb:4: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 4 of the file rookie_scraper.ipynb. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  "cell_type": "code",
rookie_scraper.ipynb:4: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 4 of the file rookie_scraper.ipynb. To get rid of this warning, pass the additional argument 'features="lxml"' to the B

Below is a parser that helps create an easy html lookup from basketball reference, the parser will parse the first letter of the last name (lowercase), and create the id that will help with the html lookup

In [14]:
def roy_parser(name):
    fn_ln= name.split(" ")
    first_letter_of_last_name= fn_ln[1][:1].lower()
    first_two_letters_of_first_name= fn_ln[0][:2].lower()
    last_name_lower= fn_ln[1][:5].lower()
    html_look_up= last_name_lower + first_two_letters_of_first_name + "01"
    return first_letter_of_last_name,html_look_up

In [51]:
def get_stats(hyperlink):
    url= hyperlink
    html = urlopen(url)
    soup = BeautifulSoup(html)
    
    headers = [th.getText() for th in soup.findAll('tr',limit=2)[0].findAll('th')]
    headers = headers[1:]
    
    rows = soup.findAll('tr')[1:]
    player_stats = [[td.getText() for td in rows[i].findAll('td')] for i in range(len(rows))]
    stats = pd.DataFrame(player_stats, columns = headers)
    if stats.shape[0]>5:
        stats= stats.head()
    num_stats= list(stats.columns.values)[4:]
    #converting numerical stat values to floats
    for stat in num_stats:
        stats[stat]= pd.to_numeric(stats[stat], errors='coerce')
    return stats.drop_duplicates().dropna(how='all')

In [52]:
roys={}
for i in range(len(links_of_roy_votes)):
    roys[names_of_roy_votes[i]]= get_stats(links_of_roy_votes[i])

In [53]:
def get_advanced_stats(hyperlink):
    url= hyperlink + '#all_advanced'
    html = urlopen(url)
    soup = BeautifulSoup(html)
    
    headers = [th.getText() for th in soup.findAll('tr',limit=2)[0].findAll('th')]
    headers = headers[1:]
    
    rows = soup.findAll('tr')[1:]
    player_stats = [[td.getText() for td in rows[i].findAll('td')] for i in range(len(rows))]
    stats = pd.DataFrame(player_stats, columns = headers)
    return stats.drop_duplicates().dropna(how='all')

In [59]:
pd_r_s=[]
pd_r_five=[]
ad_r_s=[]
ad_r_five=[]
rd_r_s=[]
rd_r_five=[]
fgd_r_s=[]
fgd_r_five=[]
for index in range(len(names_of_roy_votes)):
    pd_r_s.append(roys[names_of_roy_votes[index]]['PTS'][1]-roys[names_of_roy_votes[index]]['PTS'][0])
    pd_r_five.append(roys[names_of_roy_votes[index]]['PTS'].mean()-roys[names_of_roy_votes[index]]['PTS'][0])
    ad_r_s.append(roys[names_of_roy_votes[index]]['AST'][1]-roys[names_of_roy_votes[index]]['AST'][0])
    ad_r_five.append(roys[names_of_roy_votes[index]]['AST'].mean()-roys[names_of_roy_votes[index]]['AST'][0])
    rd_r_s.append(roys[names_of_roy_votes[index]]['TRB'][1]-roys[names_of_roy_votes[index]]['TRB'][0])
    rd_r_five.append(roys[names_of_roy_votes[index]]['TRB'].mean()-roys[names_of_roy_votes[index]]['TRB'][0])
    fgd_r_s.append(roys[names_of_roy_votes[index]]['FG%'][1]-roys[names_of_roy_votes[index]]['FG%'][0])
    fgd_r_five.append(roys[names_of_roy_votes[index]]['FG%'].mean()-roys[names_of_roy_votes[index]]['FG%'][0])

In [62]:
roys['Michael Jordan']

,Age,Tm,Lg,Pos,G,GS,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,21,CHI,NBA,SG,82,82,38.3,10.2,19.8,0.515,...,0.845,2.0,4.5,6.5,5.9,2.4,0.8,3.5,3.5,28.2
1,22,CHI,NBA,SG,18,7,25.1,8.3,18.2,0.457,...,0.840,1.3,2.3,3.6,2.9,2.1,1.2,2.5,2.6,22.7
2,23,CHI,NBA,SG,82,82,40.0,13.4,27.8,0.482,...,0.857,2.0,3.2,5.2,4.6,2.9,1.5,3.3,2.9,37.1
3,24,CHI,NBA,SG,82,82,40.4,13.0,24.4,0.535,...,0.841,1.7,3.8,5.5,5.9,3.2,1.6,3.1,3.3,35.0
4,25,CHI,NBA,SG,81,81,40.2,11.9,22.2,0.538,...,0.850,1.8,6.2,8.0,8.0,2.9,0.8,3.6,3.0,32.5
